# Visualizing spatio-chemical filters with pythreejs


## Important: Cells must be executed one at time (rather than with Cell -> Run All)



# Calculate filter specificities (precomputed)
The first step is to calculate the filter specificities.
For the atomic filters, we must simply extract the neighborhood embedding layer parameters from the keras model object.
For the amino acid filters, since the amino-acid specificity of individual gaussian components is *non-linear*, we must first determine it on a set of test proteins. This is done as follows:
1. Calculate, for each gaussian component of each filter, the distribution of activities.
2. Identify the top-1/5\% activating residues.
3. Determine their amino acid type, secondary structure and accessible surface area.
4. Determine the mean activity across the top-activating residues.

For the trained networks, the specificities are precomputed. Otherwise, calculating the amino acid filter specificities takes about 1-2 hour on a laptop
To avoid this step and visualize only the atom filters, specify only_atom=True

Then, for a given filter, display the gaussians with highest mean activity (threshold1 = 33\% of the maximum mean activity), and show attribute specificity as inset.



In [1]:
from visualizations import show_3d_filters,weight_logo_3d
from utilities import dataset_utils
import numpy as np

mode = 'epitope' # Prediction mode: 'interface' (protein-protein binding sites), 'epitope' (B-cell epitopes), 'idp' (protein - intrinsically disordered proteins binding sites)
use_MSA = True # Whether to use evolutionary information or not.



if mode == 'interface':
    top_percent = 5
    if use_MSA:
        model_name = 'ScanNet_PPI'
    else:
        model_name = 'ScanNet_PPI_noMSA'
    dataset_name = 'PPBS_validation'
    list_origins = np.concatenate([dataset_utils.read_labels('datasets/PPBS/labels_%s.txt'%dataset)[0]
     for dataset in ['validation_70','validation_homology','validation_topology','validation_none']
        ])
    
elif mode == 'epitope':
    top_percent = 5
    if use_MSA:
        model_name = 'ScanNet_PAI_0'
    else:
        model_name = 'ScanNet_PAI_noMSA_0'
    dataset_name = 'BCE_fold1'
    list_origins = dataset_utils.read_labels('datasets/BCE/labels_fold1.txt')[0]
    
elif mode == 'idp':
    top_percent = 5
    if use_MSA:
        model_name = 'ScanNet_PIDPI_0'
    else:
        model_name = 'ScanNet_PIDPI_noMSA_0'
    dataset_name = 'PIDPBS_fold0'
    list_origins = None
#     list_origins = dataset_utils.read_labels('datasets/PIDP/labels_fold0.txt')[0]
        
    


filter_specificities = show_3d_filters.calculate_filter_specificities(
    model_name,
    dataset_name = dataset_name,
    dataset_origins = list_origins,
    biounit=False,
    ncores=4,
    only_atom=False,
    top_percent = top_percent,
    fresh = False,
    Lmax = 1024

)




Using TensorFlow backend.
/opt/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/py36/lib/python3.6/site-packag

## Instantiate sphere geometry
This cell must be executed first

In [2]:
sg = weight_logo_3d.make_sphere_geometry(30)

# Interactive visualization of one amino acid filter

In [26]:
renderer = show_3d_filters.plot_aminoacid_filter(filter_specificities,117,sg=sg);
display(renderer)

Renderer(camera=PerspectiveCamera(position=(19.200000000000003, 12.0, 19.200000000000003), projectionMatrix=(1…

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

...

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

# Interactive visualization of one atomic filter

In [ ]:
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,119,sg=sg,threshold1=0.33);
display(renderer)

# Filter visualization with custom camera position

In [ ]:
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,30,sg=sg,
                                             camera_position=[-0.3, 0.6, 1.0]);

display(renderer)


# Filter visualization with custom camera position, take screenshot

In [ ]:
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,119,sg=sg,
                                             camera_position=[-0.3, 0.6, 1.0]);
recorder=weight_logo_3d.make_screenshot(renderer,'screenshot_filter1.png')
display(renderer)
recorder

# Visualize atomic neighborhood

In [16]:
from visualizations import show_3d_neighborhoods
pdbid = '7jvb' # Spike protein RBD
modelid = 0
chainid = 'A'
residue = 493 # ACE2 binding site.
atom = 'N'

    
atom_positions,atom_types,atom_bonds = show_3d_neighborhoods.get_neighborhood(
        pdb = pdbid[:4],
        model = modelid,
        chain = chainid,
        resnumber = residue,
        atom = atom,
        assembly=False,
        biounit=False,
)

renderer = show_3d_neighborhoods.show_atoms(atom_positions,atom_types,atom_bonds,render=True,
                                               radius_scale = 0.15,show_frame=True,
                                            camera_position=[-0.3, 0.6, 1.0]);
recorder=weight_logo_3d.make_screenshot(renderer,'screenshot_atomicneighborhood1.png')
display(renderer)
recorder

Parsing /Users/jerometubiana/PDB/pdb7jvb.ent


Renderer(camera=PerspectiveCamera(position=(-3.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, …

ImageRecorder(autosave=True, filename='screenshot_atomicneighborhood1.png', image=Image(value=b''), recording=…

# Atomic neighborhood superimposed with filter

In [14]:
from visualizations import show_3d_neighborhoods
pdbid = '7jvb' # Spike protein RBD
modelid = 0
chainid = 'A'
residue = 493 # ACE2 binding site.
atom = 'N'
filter_index =56

    
atom_positions,atom_types,atom_bonds = show_3d_neighborhoods.get_neighborhood(
        pdb = pdbid[:4],
        model = modelid,
        chain = chainid,
        resnumber = residue,
        atom = atom,
        assembly=False,
        biounit=False,
)


list_objects = show_3d_neighborhoods.show_atoms(atom_positions,atom_types,atom_bonds,render=False,
                                               radius_scale = 0.15)
renderer = show_3d_filters.plot_atomic_filter(filter_specificities,
                                                 filter_index,
                                                  y_offset = 0.25,
                                                 sg=sg,
                                                 list_additional_objects=list_objects,
                                                threshold1=0.33);

display(renderer)

Parsing /Users/jerometubiana/PDB/pdb7jvb.ent


Renderer(camera=PerspectiveCamera(position=(8.0, 5.0, 8.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.…

<Figure size 288x576 with 0 Axes>

<Figure size 288x576 with 0 Axes>

# Amino acid neighborhood

In [28]:
from visualizations import show_3d_neighborhoods
pdbid = '7jvb' # Spike protein RBD
modelid = 0
chainid = 'A'
residue = 493 # ACE2 binding site.
MSA_file = '/Users/jerometubiana/Downloads/MSA_7jvb_A_0_A.fasta' # None

    
aa_positions,aa_types,aa_bonds = show_3d_neighborhoods.get_neighborhood_aa(
        pdb = pdbid[:4],
        model = modelid,
        chain = chainid,
        resnumber = residue,
        assembly=False,
        biounit=False,
        MSA_file=MSA_file
)

renderer = show_3d_neighborhoods.show_aminoacids(aa_positions,aa_types,aa_bonds,render=True,
                                                 sg=sg);
recorder=weight_logo_3d.make_screenshot(renderer,'screenshot_aminoacidneighborhood1.png')
display(renderer)
recorder

Parsing /Users/jerometubiana/PDB/pdb7jvb.ent


Renderer(camera=PerspectiveCamera(position=(19.200000000000003, 12.0, 19.200000000000003), projectionMatrix=(1…

ImageRecorder(autosave=True, filename='screenshot_aminoacidneighborhood1.png', image=Image(value=b''), recordi…

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

...

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

# Amino acid neighborhood with filter superimposed

In [27]:
from visualizations import show_3d_neighborhoods
pdbid = '7jvb' # Spike protein RBD
modelid = 0
chainid = 'A'
residue = 490 # ACE2 binding site.
MSA_file = '/Users/jerometubiana/Downloads/MSA_7jvb_A_0_A.fasta' # None

    
aa_positions,aa_types,aa_bonds = show_3d_neighborhoods.get_neighborhood_aa(
        pdb = pdbid[:4],
        model = modelid,
        chain = chainid,
        resnumber = residue,
        assembly=False,
        biounit=False,
        MSA_file=MSA_file
)

list_objects = show_3d_neighborhoods.show_aminoacids(aa_positions,aa_types,aa_bonds,sg=sg,render=False);

renderer = show_3d_filters.plot_aminoacid_filter(filter_specificities,
                                                 117,
                                                 sg=sg,
                                                 list_additional_objects=list_objects,
                                                 threshold1=0.33,scale=3.0);
display(renderer)

Parsing /Users/jerometubiana/PDB/pdb7jvb.ent


/Users/jerometubiana/Documents/GitHub/ScanNet/visualizations/weight_logo_3d.py:373: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1,nplots,figsize=figsize)


Renderer(camera=PerspectiveCamera(position=(19.200000000000003, 12.0, 19.200000000000003), projectionMatrix=(1…

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

...

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

In [31]:
importlib.reload(show_3d_neighborhoods)

<module 'visualizations.show_3d_neighborhoods' from '/Users/jerometubiana/Documents/GitHub/ScanNet/visualizations/show_3d_neighborhoods.py'>

In [36]:
renderer = show_3d_filters.plot_aminoacid_filter(filter_specificities,112,sg=sg,scale=3.0,
                                                camera_position=[-0.3,1.0,1.0]);
recorder=weight_logo_3d.make_screenshot(renderer,'screenshot_aminoacidfilter2.png')
display(renderer)
recorder

Renderer(camera=PerspectiveCamera(position=(-7.199999999999999, 24.0, 24.0), projectionMatrix=(1.0, 0.0, 0.0, …

ImageRecorder(autosave=True, filename='screenshot_aminoacidfilter2.png', image=Image(value=b''), recording=Tru…

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

...

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>

<Figure size 144x576 with 0 Axes>